In [1]:
using BenchmarkTools
using ReverseDiff: JacobianTape, JacobianConfig, jacobian, jacobian!, compile



In [8]:
function airframe_source!(spl, pyna_ip, settings, ac, f, M_0, mu_0, c_0, rho_0, theta, phi, theta_flaps, I_landing_gear)
    
    # HSR calibration
    if settings["airframe_hsr_calibration"]
        hsr_supp = pyna_ip.f_hsr_supp.(f, theta*ones(eltype(theta_flaps), (settings["n_frequency_bands"], )))
    end
    
    # Add airframe noise components
    if "wing" in ac.comp_lst
        trailing_edge_wing_fwd!(spl, settings, ac, f, M_0, c_0, rho_0, mu_0, theta, phi, hsr_supp)
    end
    if "tail_v" in ac.comp_lst
        trailing_edge_vertical_tail_fwd!(spl, settings, ac, f)
    end
    if "tail_h" in ac.comp_lst
        trailing_edge_horizontal_tail_fwd!(spl, x, settings, ac, f)
    end
    if "les" in ac.comp_lst
        leading_edge_slat_fwd!(spl, x, settings, ac, f)
    end
    if "tef" in ac.comp_lst
        trailing_edge_flap_fwd!(spl, x, settings, ac, f)
    end
    if "lg" in ac.comp_lst
        landing_gear_fwd!(spl, x, settings, ac, f)
    end
end

airframe_source! (generic function with 1 method)

In [85]:
# Inputs 
M_0 = 0.3
c_0 = 300.
rho_0 = 1.2
mu_0 = 1e-5
theta = 30.
phi = 10.
hsr_supp = ones(24)
theta_flaps = 10.
I_landing_gear = 1
x_af = vcat(M_0, c_0, theta, phi, I_landing_gear)
y_af = zeros(24)

f = [50.11,63.09,79.43,100.,125.89,158.49,199.53,251.19,316.23,398.11,501.19,630.96,794.33,1000.,  
     1258.93,1584.89,1995.26,2511.89,3162.28,3981.07,5011.87,6309.57,7943.28,10000.]

settings = Dict()
settings["r_0"] = 0.3048
settings["p_ref"] = 2e-5
settings["n_frequency_bands"] = 24
settings["n_frequency_subbands"] = 5

struct Aircraft
    mtow
    n_eng
    comp_lst
    af_S_h
    af_S_v
    af_S_w
    af_b_f
    af_b_h
    af_b_v
    af_b_w
    af_S_f
    af_s
    af_d_mg
    af_d_ng
    af_l_mg
    af_l_ng
    af_n_mg
    af_n_ng
    af_N_mg
    af_N_ng
    c_d_g
    mu_r
    B_fan
    V_fan
    RSS_fan
    M_d_fan
    inc_F_n
    TS_lower
    TS_upper
    af_clean_w
    af_clean_h
    af_clean_v
    af_delta_wing
end
ac = Aircraft(55000.0,3,["wing", "tail_h", "les", "tef", "lg"],   20.16,21.3677,150.41, 6.096,5.6388,4.7244,20.51304,11.1484,1.0,0.9144,0.82296,2.286,1.8288,4.0,  2.0,  2.0,  1.0,0.0240,  0.0175,25,  48,  300.0,1.68,0.25, 0.65,  1.0,  true,false, true,  true)

Aircraft(55000.0, 3, ["wing", "tail_h", "les", "tef", "lg"], 20.16, 21.3677, 150.41, 6.096, 5.6388, 4.7244, 20.51304, 11.1484, 1.0, 0.9144, 0.82296, 2.286, 1.8288, 4.0, 2.0, 2.0, 1.0, 0.024, 0.0175, 25, 48, 300.0, 1.68, 0.25, 0.65, 1.0, true, false, true, true)

In [86]:
@time landing_gear_fwd!(y_af, x_af)

  1.091747 seconds (2.25 M allocations: 114.200 MiB, 4.83% gc time)


24-element Array{Float64,1}:
  45.885689291848244
  71.21617153670186
 108.1349247643244
 159.30931418631178
 225.97491987747847
 306.336850569562
 391.8743094108835
 465.44505717327047
 516.7725225046667
 536.8426073910209
 519.0222926709895
 468.1701865096901
 397.8209085319573
 323.16030390206157
 255.05892156028537
 198.37008398930936
 153.54096763112318
 118.95182612468709
  92.47872390846595
  72.19940124563217
  56.58904381187391
  44.50069774130668
  35.086340882895236
  27.719019466267824

In [88]:
Y = zeros(24)
X = zeros(29)
J = Y.*X'

const h_tape = JacobianTape(landing_gear_fwd!, Y, X)
const compiled_h_tape = compile(h_tape)

ReverseDiff.CompiledTape(#35)

In [89]:
@time jacobian!(J, compiled_h_tape, x_af)

  2.997553 seconds (7.40 M allocations: 351.534 MiB, 3.27% gc time)


24×29 Array{Float64,2}:
   730.73   -0.297356    2.44059  …   0.0     0.0      0.0      0.0
  1158.89   -0.443174    3.76843      0.0     0.0      0.0      0.0
  1814.69   -0.632182    5.67877      0.0     0.0      0.0      0.0
  2779.87   -0.852615    8.28267      0.0     0.0      0.0      0.0
  4125.97   -1.07408    11.6051       0.0     0.0      0.0      0.0
  5893.72   -1.23366    15.4962   …   0.0     0.0      0.0      0.0
  8057.54   -1.19461    19.4162       0.0     0.0      0.0      0.0
 10248.5    -0.916883   22.5287       0.0     0.0      0.0      0.0
 12018.9    -0.544085   24.5103       0.0     0.0      0.0      0.0
 13223.7    -0.0189158  24.8825       0.0     0.0      0.0      0.0
 13515.0     0.522205   23.483    …   0.0     0.0      0.0      0.0
 12791.4     0.915593   20.7105       0.0     0.0      0.0      0.0
 11281.4     1.08303    17.2748       0.0     0.0      0.0      0.0
  9396.91    1.05205    13.85         0.0     0.0      0.0      0.0
  7522.18    0.90846    